# New-RscMssqlLogShippingSecondary
DBAs are familiar with SQL Server native log shipping. It is generally a 3 step process where a log backup is taken on one server, copied to another server, and restored on that second server. Rubrik has log shipping built into the product in a similar workflow. The SLA will take a log backup and shortly after the log backup is complete, it is automatically recovered to the secondary server. 

New-RscMssqlLogShippingSecondary has multiple parameters, but the important ones to understand for most workflows are:
- RscMssqlDatabase: Database object returned from Get-RscMssqlDatabase
- RscCluster: Database object returned from Get-RscCluster
- TargetMssqlInstance: SQL Server Instance Object returned from Get-RscMssqlInstance
- TargetDatabaseName: Any name you want to call your database when it is recovered onto the target instance
- TargetDataPath: Single path that all data files will be placed into 
- TargeLogPath: Single path that all log files will be placed into
- TargetFilePaths: Object can be built manually like the below example, or by using Get-RscMssqlDatabaseFiles. 
- MaxDataStreams: This controls the number of streams used for the restore. By default, Rubrik will use 2 streams. This cannot exceed a value of 8. 

    In general, the default value of 2 performs best. However in some cases, increasing the value can provide better performance of the restore. Do not change this value in a
    production setting without running some tests in a non-production environment. 
- DisconnectStandbyUsers: Automatically disconnect users when restoring backups
- State: State of Log Shipping Secondaary. Can be either RESTORING or STANDBY
- AutomaticReseed: Automatically reseed the log shipping configuration when the primary transaction log chain breaks 

# Workflow
To add a log shipping secondary of a database, you need to do the following
1. Use **Connect-RSC** to Connect to RSC
1. Use **Get-RscCluster** to get the cluster id that the SQL Server is registered. This is akin to the datacenter that the SQL Server is located in. While you can run the next command without the cluster ID, you may get more records back than expected. This is because your SQL Server Host could exist on 2 different cluster. Once as a replicated source and once as the replicated target. 
1. Use **Get-RscMssqlInstance** to get the instance id of the SQL Server based on the name of the instance and the cluster where it is located. 
1. Use **Get-RscMssqlDatabase** to get the database ID based on the database name and the SQL Server Instance. 
1. Use **Get-RscMssqlDatabaseFiles** to get the list of files from the backup. 
1. Use **Get-RscMssqlInstance** to get the instance id of the Target SQL Server based on the name of the instance and the cluster where it is located. 
1. Use **NNew-RscMssqlLogShippingSecondary** to start an in place restore of the database to the point in time provided. 
1. Use **Get-RscRequest** to track the request and know when it completes. ***DOES NOT EXIST YET***
1. Use **Disconnect-RSC** to disconnect from RSC

## Connect to RSC

In [ ]:
Set-Location '/Users/Chris.Lumnah/Library/CloudStorage/GoogleDrive-chris.lumnah@rubrik.com/My Drive/1.Projects/Powershell SDK/rubrik-powershell-sdk'
Import-Module ./output/rubriksecuritycloud -force
# Import-Module RubrikSecurityCloud
Connect-RSC

## Get the Cluster ID

In [ ]:
$ClusterName = "vault-r-london"
$RscCluster = Get-RscCluster -Name $ClusterName

## Get the Instance Information

In [ ]:
$HostName = "rp-sql19s-001.perf.rubrik.com"
$RscHost = Get-RscHost -Name $HostName -OsType Windows
$RscMssqlInstance = Get-RscMssqlInstance -RscHost $RscHost -RscCluster $RscCluster

## Get the Database Information

In [ ]:
$DatabaseName = "AdventureWorks2019"
$RscMssqlDatabase = Get-RscMssqlDatabase -Name $DatabaseName  -RscMssqlInstance $RscMssqlInstance

## Get the Target Instance Information

In [ ]:
$HostName = "rp-sql19s-001.perf.rubrik.com"
$RscHost = Get-RscHost -Name $HostName -OsType Windows
$TargetMssqlInstance = Get-RscMssqlInstance -RscHost $RscHost -RscCluster $RscCluster

## Add log shipping secondary using the Simple Method

In [ ]:
$NewRscMssqlLogShippingSecondary = @{
    RscMssqlDatabase = $RscMssqlDatabase
    RscCluster = $RscCluster
    TargetMssqlInstance = $TargetMssqlInstance
    TargetDatabaseName = "test"
    TargetDataPath = "c:\mnt\sqldata\"
    TargeLogPath = "c:\mnt\sqllogs\"
    State = "RESTORING"
}
New-RscMssqlLogShippingSecondary @NewRscMssqlLogShippingSecondary

## Add log shipping secondary using the Advanced Method
### Get the fully formatted recovery point

In [ ]:
$RecoveryDateTime = Get-RscMssqlDatabaseRecoveryPoint -RscMssqlDatabase $RscMssqlDatabase -Latest

### Get list of files of the database from the backup

In [ ]:
$RscMssqlDatabaseFiles = Get-RscMssqlDatabaseFiles -RscMssqlDatabase $RscMssqlDatabase -RecoveryDateTime $RecoveryDateTime
$RscMssqlDatabaseFiles

### Create hash table of files

In [ ]:
$TargetFilePaths = @()
foreach($File in $RscMssqlDatabaseFiles){
    If ($File.FileType -eq 'MSSQL_DATABASE_FILE_TYPE_DATA'){
        $TargetDataPath = @{
            exportPath = $File.OriginalPath
            logicalName = $File.LogicalName
            newFilename = "Lumnah_Test.mdf"
        }
        $TargetFilePaths += $TargetDataPath
    }

    If ($File.FileType -eq 'MSSQL_DATABASE_FILE_TYPE_LOG'){
        $TargetLogPath = @{
            exportPath = $File.OriginalPath
            logicalName = $File.LogicalName
            newFilename = "Lumnah_Test_log.ldf"
        } 
        $TargetFilePaths += $TargetLogPath
    }
}
$TargetFilePaths

In [ ]:
$NewRscMssqlLogShippingSecondary = @{
    RscMssqlDatabase = $RscMssqlDatabase
    RscCluster = $RscCluster
    TargetMssqlInstance = $TargetMssqlInstance
    TargetDatabaseName = "test"
    TargetFilePaths = $TargetFilePaths
    State = "RESTORING"
}
New-RscMssqlLogShippingSecondary @NewRscMssqlLogShippingSecondary

## Track the status of the restore

---

# Disconnect from RSC
It is important to have each script you create end by disconnecting from RSC. If you do not, then you will run into issues with token authentication exhaustion. RSC allows a Service Account to have 10 concurrent open connections to RSC. Additional requests will be denied until existing connections are closed. 

You can easily disconnect your connection by running the below statement. 

In [ ]:
Disconnect-Rsc